# WAF Auto App - Complete Deployment

This notebook performs an end-to-end deployment:
1. Deploys the WAF Assessment dashboard to your workspace
2. Gets the published dashboard URL
3. Configures the waf-auto app with the dashboard URL
4. Deploys the waf-auto app
5. Returns the app URL where the dashboard is embedded in an iframe

**Prerequisites:**
- Clone this repo to Databricks Repos
- Or upload the dashboard JSON files to the workspace


In [ ]:
# Install required packages
%pip install databricks-sdk --quiet
dbutils.library.restartPython()


In [ ]:
import json
import os
import time
from pathlib import Path
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.workspace import ImportFormat

# Initialize workspace client
w = WorkspaceClient()

# Get workspace info
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
workspace_url = f"https://{workspace_url}"
user = w.current_user.me()

print(f"✓ Connected to: {workspace_url}")
print(f"✓ User: {user.user_name}")
print()


## Step 1: Deploy WAF Dashboard


In [ ]:
# Configuration
DASHBOARD_DISPLAY_NAME = "WAF Assessment Dashboard"
DASHBOARD_FOLDER = "/Workspace/Shared/WAF-Assessment"

# Path to dashboard JSON - adjust based on your repo structure
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
repo_root = str(Path(notebook_path).parent.parent)  # Go up to repo root
dashboard_file = f"{repo_root}/dashboards/WAF_ASSESSMENTv2.1.lvdash.json"

print(f"Repository root: {repo_root}")
print(f"Dashboard file: {dashboard_file}")
print(f"Target folder: {DASHBOARD_FOLDER}")
print()


In [ ]:
def load_dashboard_from_workspace(workspace_path):
    """Load dashboard JSON from workspace file"""
    try:
        content = w.workspace.export(workspace_path, format=ImportFormat.SOURCE)
        return json.loads(content.content.decode('utf-8'))
    except Exception as e:
        print(f"Error loading {workspace_path}: {e}")
        return None

def deploy_dashboard(display_name, workspace_path, parent_folder):
    """Deploy dashboard and return its URL"""
    print(f"📊 Deploying: {display_name}")
    
    # Load dashboard data
    dashboard_data = load_dashboard_from_workspace(workspace_path)
    if not dashboard_data:
        raise Exception(f"Could not load dashboard from {workspace_path}")
    
    # Create folder
    try:
        w.workspace.mkdirs(parent_folder)
    except:
        pass
    
    # Check if dashboard exists
    existing_id = None
    for dash in w.lakeview.list():
        if dash.display_name == display_name:
            existing_id = dash.dashboard_id
            break
    
    serialized = json.dumps(dashboard_data)
    
    if existing_id:
        print(f"   → Updating existing dashboard (ID: {existing_id})")
        w.lakeview.update(
            dashboard_id=existing_id,
            display_name=display_name,
            serialized_dashboard=serialized
        )
        dash_id = existing_id
    else:
        print(f"   → Creating new dashboard")
        result = w.lakeview.create(
            display_name=display_name,
            parent_path=f"{parent_folder}/{display_name}",
            serialized_dashboard=serialized
        )
        dash_id = result.dashboard_id
    
    # Publish dashboard
    print(f"   → Publishing dashboard")
    w.lakeview.publish(dashboard_id=dash_id)
    
    # Get org ID for the URL
    org_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("orgId").get()
    
    dashboard_url = f"{workspace_url}/dashboardsv3/{dash_id}?o={org_id}"
    
    print(f"   ✅ Dashboard deployed!")
    print(f"   Dashboard ID: {dash_id}")
    print(f"   URL: {dashboard_url}")
    
    return {
        'id': dash_id,
        'url': dashboard_url,
        'org_id': org_id
    }

print("✓ Functions defined")


In [ ]:
# Deploy the dashboard
dashboard_info = deploy_dashboard(
    display_name=DASHBOARD_DISPLAY_NAME,
    workspace_path=dashboard_file,
    parent_folder=DASHBOARD_FOLDER
)

print("\n" + "="*70)
print("✅ Dashboard deployment complete!")
print("="*70)
print(f"Dashboard URL: {dashboard_info['url']}")
print()


## Step 2: Deploy WAF Auto App

Now we'll deploy the app using Databricks Apps.
The app will embed the dashboard URL in an iframe.


In [ ]:
APP_NAME = "wafauto"
APP_DESCRIPTION = "Databricks WAF Dashboard App - Embedded Assessment Dashboard"

def deploy_app():
    """Deploy the waf-auto app using Databricks Apps REST API"""
    print(f"🚀 Deploying {APP_NAME} app...")
    
    try:
        # Get API token and headers
        api_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
        headers = {
            "Authorization": f"Bearer {api_token}",
            "Content-Type": "application/json"
        }
        
        # App source path in the repo
        app_source_path = f"{repo_root}/waf-auto"
        
        print(f"   → Configuring app with dashboard URL...")
        
        # Prepare app configuration
        app_config = {
            "name": APP_NAME,
            "description": APP_DESCRIPTION,
            "source_code_path": app_source_path
        }
        
        # Check if app already exists
        print(f"   → Checking for existing app...")
        list_url = f"{workspace_url}/api/2.0/preview/apps"
        list_response = w.api_client.do('GET', list_url)
        
        existing_app = None
        if 'apps' in list_response:
            for app in list_response['apps']:
                if app.get('name') == APP_NAME:
                    existing_app = app
                    break
        
        if existing_app:
            print(f"   → Updating existing app (ID: {existing_app['name']})")
            # Update the app
            update_url = f"{workspace_url}/api/2.0/preview/apps/{APP_NAME}"
            w.api_client.do('PATCH', update_url, data=app_config)
        else:
            print(f"   → Creating new app...")
            # Create new app
            create_url = f"{workspace_url}/api/2.0/preview/apps"
            w.api_client.do('POST', create_url, data=app_config)
        
        print(f"   ✅ App configured!")
        
        # Deploy the app
        print(f"   → Deploying app...")
        deploy_url = f"{workspace_url}/api/2.0/preview/apps/{APP_NAME}/deployments"
        deploy_config = {
            "source_code_path": app_source_path,
            "mode": "SNAPSHOT"
        }
        w.api_client.do('POST', deploy_url, data=deploy_config)
        
        print(f"   → Waiting for deployment...")
        time.sleep(15)  # Wait for deployment to initialize
        
        # Start the app
        print(f"   → Starting app...")
        start_url = f"{workspace_url}/api/2.0/preview/apps/{APP_NAME}/start"
        w.api_client.do('POST', start_url)
        
        # Wait for app to be running
        print(f"   → Waiting for app to start...")
        max_wait = 180
        start_time = time.time()
        
        while time.time() - start_time < max_wait:
            try:
                status_url = f"{workspace_url}/api/2.0/preview/apps/{APP_NAME}"
                status = w.api_client.do('GET', status_url)
                if status.get('state') == 'RUNNING':
                    print(f"   ✅ App is running!")
                    break
            except:
                pass
            time.sleep(10)
        
        # Get app URL
        app_url = f"{workspace_url}/apps/{APP_NAME}"
        
        print(f"   ✅ App deployed successfully!")
        print(f"   📱 App URL: {app_url}")
        
        return {
            'name': APP_NAME,
            'url': app_url,
            'status': 'deployed',
            'dashboard_url': dashboard_info['url'],
            'embed_url': dashboard_info['url'].replace('/dashboardsv3/', '/embed/dashboardsv3/')
        }
        
    except Exception as e:
        print(f"   ⚠️ Automated deployment encountered an issue: {str(e)}")
        print(f"   ")
        print(f"   📋 Manual Deployment Instructions:")
        print(f"   =====================================")
        print(f"   1. Go to: {workspace_url}/apps")
        print(f"   2. Click 'Create App'")
        print(f"   3. Choose 'App from Git repository'")
        print(f"   4. Repository: https://github.com/AbhiDatabricks/Databricks-WAF-Light-Tooling")
        print(f"   5. Branch: waf_auto_app")
        print(f"   6. Source path: waf-auto")
        print(f"   7. In Configuration, add environment variable:")
        print(f"      Name: DASHBOARD_URL")
        print(f"      Value: {dashboard_info['url']}")
        print(f"   8. Click 'Create' then 'Deploy'")
        print(f"   ")
        
        # Return manual deployment info with embed URL
        embed_url = dashboard_info['url'].replace('/dashboardsv3/', '/embed/dashboardsv3/')
        iframe_code = f'''<iframe
  src="{embed_url}"
  width="100%"
  height="600"
  frameborder="0">
</iframe>'''
        
        print(f"   📝 Dashboard Embed URL:")
        print(f"   {embed_url}")
        print(f"   ")
        print(f"   📝 Dashboard iframe code:")
        print(iframe_code)
        
        return {
            'name': APP_NAME,
            'url': f"{workspace_url}/apps",
            'status': 'manual_deployment_needed',
            'dashboard_url': dashboard_info['url'],
            'embed_url': embed_url,
            'iframe_code': iframe_code,
            'manual_instructions': "Follow instructions above to manually deploy the app"
        }

app_info = deploy_app()
print()


## Deployment Summary


In [ ]:
# Display final summary with clickable links
displayHTML(f"""
<div style="border: 3px solid #00aa00; padding: 30px; margin: 20px 0; border-radius: 12px; background: linear-gradient(135deg, #f0fff0 0%, #e8f8e8 100%);">
    <h2 style="color: #00aa00; margin-top: 0;">🎉 Deployment Complete!</h2>
    
    <div style="background: white; padding: 20px; margin: 20px 0; border-radius: 8px; border-left: 4px solid #0066cc;">
        <h3 style="margin-top: 0; color: #0066cc;">📊 Dashboard</h3>
        <p><strong>Name:</strong> {DASHBOARD_DISPLAY_NAME}</p>
        <p><strong>ID:</strong> <code>{dashboard_info['id']}</code></p>
        <p><strong>Direct Link:</strong><br>
        <a href="{dashboard_info['url']}" target="_blank" style="color: #0066cc; font-size: 14px;">{dashboard_info['url']}</a></p>
    </div>
    
    <div style="background: white; padding: 20px; margin: 20px 0; border-radius: 8px; border-left: 4px solid #ff6600;">
        <h3 style="margin-top: 0; color: #ff6600;">🚀 WAF Auto App</h3>
        <p><strong>Name:</strong> {app_info['name']}</p>
        <p><strong>Status:</strong> <code>{app_info['status']}</code></p>
        <p><strong>App URL (Dashboard Embedded):</strong><br>
        <a href="{app_info['url']}" target="_blank" style="color: #ff6600; font-size: 16px; font-weight: bold;">{app_info['url']}</a></p>
        <p style="color: #666; font-size: 14px; margin-top: 15px;">👆 Click above to open the app with the embedded dashboard in an iframe</p>
    </div>
    
    <div style="background: #fffef0; padding: 15px; margin: 20px 0; border-radius: 8px; border: 1px solid #ffd700;">
        <h4 style="margin-top: 0; color: #b8860b;">📝 Next Steps</h4>
        <ul style="color: #666;">
            <li>Click the <strong>App URL</strong> above to view your embedded dashboard</li>
            <li>Share the app URL with stakeholders for easy access</li>
            <li>The dashboard is embedded in an iframe with a clean header</li>
            <li>Use <code>/api/health</code> and <code>/api/config</code> endpoints for diagnostics</li>
        </ul>
    </div>
</div>
""")

# Print text version too
print("="*70)
print("DEPLOYMENT SUMMARY")
print("="*70)
print(f"\n📊 Dashboard: {dashboard_info['url']}")
print(f"🚀 App: {app_info['url']}")
print(f"\n✅ All done!")
print()
